# Analyzing Customer Cone Changes

### Getting started

You will need to obtain six files: two `as-rel` files, two `.paths` files, and two `ppdc-ases` files, one of each from each of the two dates. Put them all in the same directory as this notebook, or some other directory. Specify the target AS and the dates you want to examine.

Finally, set `GAINED` to `True` if you want the script to track customer cone gains, or set it to `False` if you want the script to track customer cone losses.

In [ ]:
import os
import bz2

TARGET = 5511 # Replace this with the AS in question

# Date format: YYYYMMDD
DATE1 = 20220601 # Replace this with the first date
DATE2 = 20220801 # Replace this with the second date

# Change this to False to examine ASes that disappeared from the given cone.
GAINED = True

# Leave this as-is if the files are in the same directory as this notebook.
# Otherwise, change this to the directory the files are in.
DIRECTORY = "."

# Generate file paths from given dates and directory
RELS1 = os.path.normpath("{}/{}.as-rel.txt.bz2".format(DIRECTORY, DATE1))
RELS2 = os.path.normpath("{}/{}.as-rel.txt.bz2".format(DIRECTORY, DATE2))
CONES1 = os.path.normpath("{}/{}.ppdc-ases.txt.bz2".format(DIRECTORY, DATE1))
CONES2 = os.path.normpath("{}/{}.ppdc-ases.txt.bz2".format(DIRECTORY, DATE2))
PATHS1 = os.path.normpath("{}/{}.paths.bz2".format(DIRECTORY, DATE1))
PATHS2 = os.path.normpath("{}/{}.paths.bz2".format(DIRECTORY, DATE2))

# Assign paths to variables depending on whether gained or lost ASes will be examined
if GAINED:
    word = 'gained'
    sm_paths = PATHS1
    lg_paths = PATHS2
    sm_cones = CONES1
    lg_cones = CONES2
    sm_rels = RELS1
    lg_rels = RELS2
else:
    word = 'lost'
    sm_paths = PATHS2
    lg_paths = PATHS1
    sm_cones = CONES2
    lg_cones = CONES1
    sm_rels = RELS2
    lg_rels = RELS1

The first thing to do is simply to find out what changed. This script finds the set of ASes that changed during the specified time interval.

In [ ]:
# Get a specific cone from a ppdc-ases file
def get_cone(filename, target):
    cone = set()
    ts = str(target)
    with bz2.open(filename, 'r') as cfile:
        for line in cfile:
            line = line.decode('utf-8')
            if line[:line.index(' ')] == ts:
                for asn in line[line.index(' ')+1:].split(' '):
                    cone.add(int(asn))
                return cone

# Obtain the target's cones at both points in time
cone_sm = get_cone(sm_cones, TARGET)
cone_lg = get_cone(lg_cones, TARGET)

# Find the set difference
changed = cone_lg.copy()
for asn in cone_sm:
    changed.discard(asn)

As explained in the introduction, paths and AS relationships are important to understand cones. This function takes a list of paths, and for each pair of adjacent ASNs, annotates the path with their relationship. The following notation is used:
- A < B means that A is a provider to B
- A > B means that A is a customer of B
- A - B means that A and B are peers

In [ ]:
# Given a paths file and an as-rel file, output paths annotated with relationship info
def annotate_paths(pathsfile, relsfile):

    # Make a dict that contains relationship information
    relsdict = {}
    with bz2.open(relsfile, 'r') as f:
        for line in f:
            line = line.decode('utf-8')
            # skip all lines that begin with '#'
            if line[0] == '#':
                continue

            # each line represents one relationship
            # store the relationship in a dict so it can be accessed quickly
            clsplit = line.split('|')
            for i in range(3):
                clsplit[i] = int(clsplit[i])
            if not clsplit[0] in relsdict:
                relsdict[clsplit[0]] = { clsplit[1]: clsplit[2] }
            else:
                relsdict[clsplit[0]][clsplit[1]] = clsplit[2]

    # Iterate through the paths file and annotate each path
    annotated_paths = []
    with bz2.open(pathsfile, 'r') as file:
        for current_line in file:
            current_line = current_line.decode()

            # Skip lines beginning with #
            if current_line[0] == '#':
                continue
            clsplit = current_line.split('|')

            # Convert ASNs to ints
            for i in range(len(clsplit)):
                clsplit[i] = int(clsplit[i])
            
            # Iterate through the path, annotating with relationships
            annotate = [clsplit[0]]
            for i in range(1, len(clsplit)):
                if clsplit[i-1] in relsdict and clsplit[i] in relsdict[clsplit[i-1]]:
                    if relsdict[clsplit[i-1]][clsplit[i]] == -1:
                        annotate.append('<')
                    elif relsdict[clsplit[i-1]][clsplit[i]] == 0:
                        annotate.append('-')
                    else:
                        annotate.append('>')
                else:
                    if clsplit[i] in relsdict:
                        if not clsplit[i-1] in relsdict[clsplit[i]]:
                            annotate.extend(['?', clsplit[i]])
                            continue
                        if relsdict[clsplit[i]][clsplit[i-1]] == -1:
                            annotate.append('>')
                        elif relsdict[clsplit[i]][clsplit[i-1]] == 0:
                            annotate.append('-')
                        else:
                            annotate.append('<')
                    else:
                        annotate.append('?')
                annotate.append(clsplit[i])
            annotated_paths.append(annotate)
    return annotated_paths

In order to infer cones, the annotated paths must be cropped. The following function crops paths to only segments with provider-to-customer relationships, where the first provider-to-customer relationship in a path is excluded unless it is preceded by a peer-to-peer relationship.

In [ ]:
# Process the given paths, cropping them based on relationships
def crop_paths(annotated_paths):
    cropped_paths = []
    for path in annotated_paths:
        new_path = []
        if '<' in path:

            # If the first < relationship is preceded by -, include it
            if path[path.index('<') - 2] == '-':
                new_path = path[(path.index('<') - 1):]

            # Otherwise, exclude it
            else:
                new_path = path[(path.index('<') + 1):]
        
        # If path doesn't have <, skip it
        else:
            continue

        # Cropped paths should consist of only < relationships,
        # so paths with - or > should be split up
        if not ('-' in new_path or '>' in new_path):
            cropped_paths.append(new_path[::2])
        else:
            to_add_outer = []
            to_add_inner = []
            for elt in new_path:
                if type(elt) == int:
                    to_add_inner.append(elt)
                else:
                    if elt == '-' or elt == '>':
                        if len(to_add_inner) > 1:
                            to_add_outer.append(to_add_inner)
                        to_add_inner = []
            cropped_paths.extend(to_add_outer)
    return cropped_paths

The above functions will process the paths so that they can be used to make inferences about cones. Since cones have to do with which ASes control access to other ASes, a helpful data structure is one that shows which ASes are in which customer cone, and how they got there. 

For example, say we have the path `A|B|C|D`. `D` is in the cone of `A`, but it's only there because `B` granted `A` access to `D`. The following function generates a data structure that represents these relationships:

`A`:
- access to `D` via `B`
- access to `C` via `B`
- access to `B` via `B`

`B`:
- access to `D` via `C`
- access to `C` via `C`

etc.

In [ ]:
# Generate a data structure using cropped paths that indicates how ASes access other ASes
def get_conduits(cpaths):
    conduits = {}
    for path in cpaths:

        # Loop through the path
        for i in range(len(path) - 1):

            # the current ASN is considered as a provider, and is a key in the (parent) dict
            # its value is a dict corresponding to it, i.e. a child dict
            current = path[i]
            if not current in conduits:
                conduits[current] = {}

            # the current ASN's direct customer is the conduit through which indirect customers pass
            conduit = path[i+1]

            # indirect customers get added as keys to the (child) dict corresponding to the provider
            for j in range(i+1, len(path)):
                sec = path[j]
                if not sec in conduits[current]:
                    conduits[current][sec] = set()
                
                # the conduit is added to a every set representing an indirect customer
                conduits[current][sec].add(conduit)
    return conduits

The above functions will now be used to generate some data that will be used in the analysis. This part will take a while.

In [ ]:
annotated_paths_sm = annotate_paths(sm_paths, sm_rels)
conduits_sm = get_conduits(crop_paths(annotated_paths_sm))
annotated_paths_lg = annotate_paths(lg_paths, lg_rels)
conduits_lg = get_conduits(crop_paths(annotated_paths_lg))

To find the cause that resulted in customer cone change, we devise a function that finds the cause for a single AS. Then, that function is applied to every AS that was gained or lost, and the found causes are aggregated to determine which cause contributed the most to overall customer cone change.

The function below initially performs a depth-first search on the graph of connections between the secondary AS (the customer) and the target AS (the one whose cone is being investigated). If there are no "broken links" (changed relationships), the function scans through the lists of which ASes announced which, looking for a failure/refusal by some intermediate AS to announce the secondary AS to the target AS.

In [ ]:
# Given a target and an AS (the "secondary target") in its cone, try to find the "broken link" 
# i.e. why it stopped/started being included in the cone. Specifically, find the link that is
# in conduits_2 but not in conduits_1
def find_broken_links(sec, conduits_1, conduits_2):
    current = 0
    all_conduits = []
    link_stack = []

    # This algorithm attempts, using a depth-first search, to determine whether the 
    # appearance/disappearance of a link in the graph of ASes caused the secondary to be 
    # included/excluded from the cone of the target AS.

    # First, the conduits that gave the target access to the secondary are added to a stack
    for asn in conduits_2[TARGET][sec]:
        link_stack.append((TARGET, asn))
    
    # The BFS proceeds by iterating through the link stack
    while len(link_stack) != 0:

        # Pop the top link
        cur_link = link_stack.pop()
        current = cur_link[0]
        conduit = cur_link[1]

        # Add further links to the top of the stack
        if conduit != sec:
            for asn in conduits_2[conduit][sec]:
                link_stack.append((conduit, asn))

        # Check whether the current link is found in conduits_1. If it isn't, it's the broken link.
        if not (current in conduits_1 and conduit in conduits_1[current]):
            return 'link changed: {} < {}'.format(current, conduit)
        if not conduit in all_conduits: 
            all_conduits.append(conduit)
    
    # If all links in conduits_2 were also found in conduits_2, we instead look for a failure
    # to announce. The highest-level conduit that could've announced the secondary, but didn't,
    # is singled out.
    for asn in all_conduits:
        if asn in conduits_1[TARGET]:
            if sec in conduits_1[asn]:
                return 'failed to announce: {}'.format(asn)
        else:
            return 'indeterminate'

Next, the above function is applied to all ASes gained/lost from the target AS's customer cone during the specified time interval, and the causes are aggregated and printed.

In [ ]:
# Make a dict that holds every cause and how many ASes that cause was responsible for
causes = {}

# Another dict to summarize the causes
causes_summary = {'failed to announce': 0, 'link changed': 0, 'indeterminate': 0}
for asn in changed:
    cause = find_broken_links(asn, conduits_sm, conduits_lg)
    
    if 'failed to announce' in cause:
        causes_summary['failed to announce'] += 1
    elif 'link changed' in cause:
        causes_summary['link changed'] += 1
    else:
        causes_summary['indeterminate'] += 1
        
    if not cause in causes:
        causes[cause] = 0
    causes[cause] += 1

# Print the top 50 causes
print('Top 50 Causes:')
n = 0
for cause, count in sorted(causes.items(), key=lambda r_c: r_c[1], reverse=True):
    print('{}, count: {}, percent of {}: {}, percent of overall cone: {}'.format(cause, count, word, str(100*count/len(changed))[:4], str(100*count/len(cone_lg))[:4]))
    n += 1
    if n > 50:
        break

# Print the summary of causes
print()
print('Summary of Causes:')
for cause, count in sorted(causes_summary.items(), key=lambda r_c: r_c[1], reverse=True):
    print('{}, count: {}, percent of {}: {}, percent of overall cone: {}'.format(cause, count, word, str(100*count/len(changed))[:5], str(100*count/len(cone_lg))[:5]))

The above results are informative, but there's a caveat: cropping paths such that the first provider-to-customer link is excluded might result in the program failing to register links that actually exist.

Since having a peer-to-peer link immediately before the first provider-to-customer link allows the provider-to-customer link to be considered, we allow for the possibility that certain newly appeared/disappeared peer-to-peer links are responsible for the change in the size of the customer cone.

First, a set is created of all the ASes that would have been in the target's cone if the paths had not been cropped. Then, the intersection of that set and the set of gained/lost ASes is found.

In [ ]:
# Get a more inclusive cone by ignoring the rules for cropping paths
def get_inclusive_target_cone(annotated_paths):
    cone = set()
    for path in annotated_paths:
        if TARGET in path:
            for elt in path[path.index(TARGET) + 1:]:
                if elt == '>' or elt == '-' or elt == '?':
                    break
                if elt == '<':
                    continue
                cone.add(int(elt))
    return cone

inclusive_targ_cone = get_inclusive_target_cone(annotated_paths_sm)

# Find the intersection of the gained/lost ASNs and the ASNs in the inclusive cone
inclusive_targ_cone_changed = inclusive_targ_cone & changed

Consequential peers are found by iterating through the set of annotated paths, looking for cases where a peer-to-peer link allowed a provider-to-customer link to be included, and checking whether the ASNs in that path are also in the set generated above.

Finally, the data obtained through this process is printed.

In [ ]:
# Make a dict of peer to peer relationships that co-occur with ASNs in the intersection found above
consequential_peers = {}
for path in annotated_paths_lg:

    # If the target is preceded by a peer, then it wouldn't have been cropped, so that path could
    # be used to find the target's cone.
    if TARGET in path and path.index(TARGET) != 0 and path[path.index(TARGET) - 1] == '-':
        current_peer = path[path.index(TARGET) - 2]
        for asn in path[path.index(TARGET) + 2::2]:

            # if an ASN from the intersection is found in the path, add it to a set that correponds
            # to the current consequential peer-to-peer link
            if asn in inclusive_targ_cone_changed:
                if not current_peer in consequential_peers:
                    consequential_peers[current_peer] = set()
                consequential_peers[current_peer].add(asn)

# Print the results of the above analysis
print("Consequential Peer-to-peer Links:")
print('(The percentages may not add up to 100% because of overlap)\n')
for k,v in sorted(consequential_peers.items(), key=lambda k_v: len(k_v[1]), reverse=True):
    print('{}-{} responsible for {} new inclusions ({}% of total)'.format(k, TARGET, len(v), str(100*len(v)/len(inclusive_targ_cone_changed))[0:4]))

Copyright (C) 2023 The Regents of the University of
California. All Rights Reserved.